In [ ]:
# Because we use dataset from kaggle, we need to upload our kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download dataset from kaggle
!kaggle datasets download -d minhhuy2810/rice-diseases-image-dataset

In [ ]:
import zipfile

# Extract zip
local_zip = '/content/rice-diseases-image-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/rice-diseases-image-dataset')
zip_ref.close()

Import neccessary packages

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [ ]:
EPOCHS = 50
LEARNING_RATE = 1e-3
BS = 32
DATA_DIR = '/content/rice-diseases-image-dataset'
LABELLED_DIR = '/content/rice-diseases-image-dataset/LabelledRice/Labelled'
TRAINING_DIR = '/content/rice-diseases-image-dataset/RiceDiseaseDataset/train'
VALIDATION_DIR = '/content/rice-diseases-image-dataset/RiceDiseaseDataset/validation'
width = 256
height = 256
depth = 3

In [ ]:
for subdir, dirs, files in os.walk(DATA_DIR):
    for file in files:
        if file == '.DS_Store':
            os.remove(os.path.join(subdir, file))

In [ ]:
# From Separated Training & Validation Folder
training_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(width, height),
    batch_size=BS,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(width, height),
    batch_size=BS,
    class_mode='categorical'
)

In [ ]:
input_shape = (height, width, depth)
chan_dim = -1
n_classes = len(os.listdir(TRAINING_DIR))
if K.image_data_format() == 'channels_first':
    input_shape = (depth, height, width)
    chan_dim = 1

In [ ]:
# Transfer Learning
pre_trained_model = tf.keras.applications.DenseNet201(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape,
    pooling='avg',
)
pre_trained_model.trainable = False
inputs = pre_trained_model.input
x = Dense(128, activation='relu')(pre_trained_model.output)
outputs = Dense(n_classes, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

Model Summary

In [ ]:
model.summary()

In [ ]:
optimizer = Adam(learning_rate=LEARNING_RATE, decay=LEARNING_RATE / EPOCHS)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

BEST_MODEL = 'saved_model/best_model'
BEST_MODEL_H5 = BEST_MODEL + '.h5'

# Save the entire model as a SavedModel.
checkpoint = ModelCheckpoint(BEST_MODEL, verbose=1, save_best_only=True, mode='auto')

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
checkpoint_h5 = ModelCheckpoint(BEST_MODEL_H5, verbose=1, save_best_only=True, mode='auto')

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BS,
    validation_data=validation_generator,
    epochs=EPOCHS,
    verbose=1,
    callbacks=[checkpoint_h5]
)

Plot the train and val curve

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()

#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
scores = model.evaluate(validation_generator)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
from tensorflow.keras.models import load_model
new_model = load_model(BEST_MODEL_H5)
scores = new_model.evaluate(validation_generator)
print(f"Best Accuracy: {scores[1]*100}")